In [7]:
#Install tqdm to have progress bar
!pip install tqdm

!pip install requests
!pip install requests-aws4auth
!pip install "sagemaker>=2.0.0<3.0.0"

!pip install install transformers[tensorflow]
!pip install -q sagemaker-experiments==0.1.13


In [1]:
#Set up ipywigets 
#%%capture
import boto3
import sagemaker
from sagemaker import get_execution_role
import numpy as np
import json




In [2]:
#Get execution roles 
##The following line WILL NOT RUN outside of sagemaker notebook! It uses the notebook IAM to do deployments and invoke endpoints. 
##This can be replaced with other IAM role setups with boto3, but this is more involved and I cannot do it on infra I do not have acess to 
role = get_execution_role()

s3_resource = boto3.resource("s3")
s3 = boto3.client('s3')

print(f'SageMaker SDK Version: {sagemaker.__version__}')

Couldn't call 'get_role' to get Role ARN from role name charlie to get Role path.


ValueError: The current AWS identity is not a role: arn:aws:iam::708362204001:user/charlie, therefore it cannot be used as a SageMaker execution role

In [4]:
#Defining default bucket for SageMaker pretrained model hosting
sagemaker_session = sagemaker.Session()

In [5]:
##########
##########

#Deploy documentation generation API 

In [9]:
from sagemaker.estimator import Estimator

hyperparameters = {"train-steps": 100}

#time.sleep(500)

estimator_unattatched = Estimator(
role=role,
instance_count=1,
instance_type="ml.c4.xlarge",
image_uri="708362204001.dkr.ecr.us-east-2.amazonaws.com/documentation-generation-api:latest",
hyperparameters=hyperparameters,
)


In [10]:
##Deploying model requires an attatched train job, 
##for first run just hack by running empty train script with random file 
estimator_unattatched.fit("<s3 path to a text file with anything in it>")

2023-01-10 20:28:20 Starting - Starting the training job...
2023-01-10 20:28:43 Starting - Preparing the instances for trainingProfilerReport-1673382500: InProgress
......
2023-01-10 20:29:43 Downloading - Downloading input data...
2023-01-10 20:30:04 Training - Downloading the training image......
2023-01-10 20:31:04 Training - Training image download completed. Training in progress..this is a training script!

2023-01-10 20:31:44 Uploading - Uploading generated training model
2023-01-10 20:31:44 Completed - Training job completed
Training seconds: 117
Billable seconds: 117


In [ ]:
###Only redeploy if new container
try:
    predictor = estimator.deploy(1, instance_type="ml.g4dn.xlarge", endpoint_name="table-qa-001")
    print("Endpoint deployment successful.")
except Exception as e:
    print("Endpoint deployment failed:", str(e))


---------------------------

In [ ]:
import boto3

# Specify the SageMaker endpoint and autoscaling configuration details
endpoint_name = 'table-qa-001'
min_capacity = 1
max_capacity = 5
target_value = 70
scale_in_cooldown = 60
scale_out_cooldown = 60

# Create a SageMaker client
sagemaker_client = boto3.client('sagemaker')

# Get the current endpoint configuration
response = sagemaker_client.describe_endpoint(EndpointName=endpoint_name)
endpoint_config_name = response['EndpointConfigName']

# Create or update the autoscaling configuration
autoscaling_config = {
    'AutoScalingEnabled': True,
    'MinCapacity': min_capacity,
    'MaxCapacity': max_capacity,
    'TargetValue': target_value,
    'ScaleInCooldown': scale_in_cooldown,
    'ScaleOutCooldown': scale_out_cooldown
}

sagemaker_client.update_endpoint(
    EndpointName=endpoint_name,
    EndpointConfigName=endpoint_config_name,
    RetainAllVariantProperties=True,
    ExcludeRetainedVariantProperties=['InitialInstanceCount', 'InitialVariantWeight'],
    InferenceSchedulerArn=response.get('InferenceSchedulerArn'),
    Tags=response.get('Tags'),
    AutoScalingType='TargetTrackingScaling',
    TargetTrackingScalingPolicyConfiguration=autoscaling_config
)

print("Autoscaling configured successfully for the SageMaker endpoint.")


The above code works for when a model does not exist but you want to deploy it for the first time. Just for posterity I have included code to instead show how to take an ECR 

hosted container, make a sagemaker model artefact from it, from that generate a new endpoint config and then update the endpoint with the new container.

In [ ]:
import boto3
# Specify the details of the ECR container
container_image_uri = 'ecr-container-image-uri'

# Create a SageMaker client
sagemaker_client = boto3.client('sagemaker')

# Create a new SageMaker model using the ECR container image
model_name = 'tableqa-tapas-0002'
primary_container = {
    'Image': container_image_uri,
    'ModelDataUrl': '<any_dummy_data_path_to_a_blank_s3_text_file' #Sagemaker wants a model arefact path but we use transformers to inject model
}
response = sagemaker_client.create_model(ModelName=model_name, ExecutionRoleArn='your-role-arn', PrimaryContainer=primary_container)

# Create a new SageMaker endpoint configuration
endpoint_config_name = 'tapas-model-endpoint-config'
response = sagemaker_client.create_endpoint_config(
    EndpointConfigName=endpoint_config_name,
    ProductionVariants=[
        {
            'VariantName': 'tapas-variant-002',
            'ModelName': model_name,
            'InitialInstanceCount': 1,
            'InstanceType': 'ml.g4dn.xlarge'  # Specify the desired instance type
        }
    ]
)

# Update the existing endpoint with the new endpoint configuration
response = sagemaker_client.update_endpoint(
    EndpointName=endpoint_name,
    EndpointConfigName=endpoint_config_name
)

print("Endpoint updated with the new configuration.")


Quick example showing how to use the sagemaker client to query our new existing endpoint

In [ ]:
import boto3
import json

# Specify the endpoint name and JSON payload
endpoint_name = 'table-qa-001'
json_payload = {"questions": ["which actor has the last name Pitt"],"table": {"Actors": ["Brad Pitt", "Leonardo Di Caprio", "George Clooney"], "Number of movies": ["87", "53", "69"]}}


# Create a SageMaker runtime client
runtime_client = boto3.client('sagemaker-runtime')

# Convert the JSON payload to a string
payload_string = json.dumps(json_payload)

# Invoke the endpoint with the JSON payload
response = runtime_client.invoke_endpoint(
    EndpointName=endpoint_name,
    ContentType='application/json',
    Body=payload_string
)

# Parse the response
response_body = response['Body'].read()
response_json = json.loads(response_body)

# Process the response as needed
print(response_json)
